In [17]:
import random
import os
import time
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import matplotlib.pyplot as plt
import numpy as np


In [34]:
class GameOfLife:
    def __init__(self,width=101,height=82):
        self.width=width
        self.height=height
        self.grid = [[0 for _ in range(width)] for _ in range(height)]
        self.next_grid = [[0 for _ in range(width)] for _ in range(height)]

    def _randomly_fill(self,probability):

        for y in range(self.height):
            for x in range(self.width):
                if random.uniform(0,1.0)<probability:
                    self.grid[y][x] = 1

    def set_pattern(self, pattern, start_x, start_y):
        """Set a specific pattern at the given position"""
        for y, row in enumerate(pattern):
            for x, cell in enumerate(row):
                grid_y = (start_y + y) % self.height
                grid_x = (start_x + x) % self.width
                self.grid[grid_y][grid_x] = cell

    def count_neighbors(self, x, y):
        #Count live neighbors of a cell using toroidal topology
        count = 0
        for dy in [-1, 0, 1]:
            for dx in [-1, 0, 1]:
                # Skip the cell itself
                if dx == 0 and dy == 0:
                    continue  
                
                # Apply toroidal wrapping
                nx = (x + dx) % self.width
                ny = (y + dy) % self.height
                
                count += self.grid[ny][nx]
        return count
    def update(self):
        #Conway's Game of Life rules
        for y in range(self.height):
            for x in range(self.width):
                neighbors = self.count_neighbors(x, y)
                
                # Apply Conway's Game of Life rules
                if self.grid[y][x] == 1:  # Living cell
                    if neighbors < 2 or neighbors > 3:
                        self.next_grid[y][x] = 0  # Cell dies
                    else:
                        self.next_grid[y][x] = 1  # Cell survives
                else:  # Dead cell
                    if neighbors == 3:
                        self.next_grid[y][x] = 1  # Cell becomes alive
                    else:
                        self.next_grid[y][x] = 0  # Cell stays dead
        
        
        self.grid, self.next_grid = self.next_grid, self.grid

    def display(self):
        """Display the grid using ASCII characters that are visually distinct"""
        os.system('cls' if os.name == 'nt' else 'clear')  # Clear console
        
        for row in self.grid:
            print(''.join('█' if cell else ' ' for cell in row))






In [35]:


Grid = List[List[int]]


@dataclass(frozen=True)
class Pattern:
    """A named Game of Life pattern represented as a 0/1 matrix."""
    name: str
    cells: Grid


class Shapes:
    """
    Library of Conway's Game of Life patterns.

    Provides:
      - get(name): return a copy of a named pattern grid
      - random(names=None): return a random pattern (Pattern object)
      - transforms: rotate / flip patterns to increase variety
    """

    def __init__(self, enable_transforms: bool = True):
        self.enable_transforms = enable_transforms
        self._patterns: Dict[str, Pattern] = self._build_patterns()

    # ---------------------------
    # Public API
    # ---------------------------

    def names(self) -> List[str]:
        return sorted(self._patterns.keys())

    def get(self, name: str, *, transform: bool = True) -> Pattern:
        if name not in self._patterns:
            raise KeyError(f"Unknown pattern: {name}. Available: {', '.join(self.names())}")

        pat = self._patterns[name]
        cells = self._copy_grid(pat.cells)

        if self.enable_transforms and transform:
            cells = self._random_transform(cells)

        return Pattern(name=pat.name, cells=cells)

    def random(self, names: Optional[List[str]] = None, *, transform: bool = True) -> Pattern:
        pool = names if names else self.names()
        if not pool:
            raise ValueError("No patterns available to choose from.")

        chosen = random.choice(pool)
        return self.get(chosen, transform=transform)

    def random_blinker_or_glider(self, *, transform: bool = True) -> Pattern:
        return self.random(names=["blinker", "glider"], transform=transform)

    # ---------------------------
    # Built-in patterns
    # ---------------------------

    def _build_patterns(self) -> Dict[str, Pattern]:
        # Note: Each pattern is a small 0/1 matrix. Your GameOfLife.set_pattern() accepts that format.
        patterns = [
            Pattern("blinker", [
                [0, 1, 0],
                [0, 1, 0],
                [0, 1, 0],
            ]),
            Pattern("glider", [
                [0, 1, 0],
                [0, 0, 1],
                [1, 1, 1],
            ]),
            Pattern("toad", [  # period-2 oscillator
                [0, 0, 0, 0],
                [0, 1, 1, 1],
                [1, 1, 1, 0],
                [0, 0, 0, 0],
            ]),
            Pattern("beacon", [  # period-2 oscillator
                [1, 1, 0, 0],
                [1, 1, 0, 0],
                [0, 0, 1, 1],
                [0, 0, 1, 1],
            ]),
            Pattern("pulsar", [  # classic period-3 oscillator (13x13)
                [0,0,0,0,0,0,0,0,0,0,0,0,0],
                [0,0,1,1,1,0,0,0,1,1,1,0,0],
                [0,0,0,0,0,0,0,0,0,0,0,0,0],
                [0,1,0,0,0,1,0,1,0,0,0,1,0],
                [0,1,0,0,0,1,0,1,0,0,0,1,0],
                [0,1,0,0,0,1,0,1,0,0,0,1,0],
                [0,0,1,1,1,0,0,0,1,1,1,0,0],
                [0,0,0,0,0,0,0,0,0,0,0,0,0],
                [0,0,1,1,1,0,0,0,1,1,1,0,0],
                [0,1,0,0,0,1,0,1,0,0,0,1,0],
                [0,1,0,0,0,1,0,1,0,0,0,1,0],
                [0,1,0,0,0,1,0,1,0,0,0,1,0],
                [0,0,0,0,0,0,0,0,0,0,0,0,0],
            ]),
            Pattern("block", [  # still life
                [1, 1],
                [1, 1],
            ]),
            Pattern("beehive", [  # still life
                [0, 1, 1, 0],
                [1, 0, 0, 1],
                [0, 1, 1, 0],
            ]),
            Pattern("loaf", [  # still life
                [0, 1, 1, 0],
                [1, 0, 0, 1],
                [0, 1, 0, 1],
                [0, 0, 1, 0],
            ]),
            Pattern("boat", [  # still life
                [1, 1, 0],
                [1, 0, 1],
                [0, 1, 0],
            ]),
            Pattern("lightweight_spaceship", [  # LWSS (moves)
                [0, 1, 1, 1, 1],
                [1, 0, 0, 0, 1],
                [0, 0, 0, 0, 1],
                [1, 0, 0, 1, 0],
            ]),
            Pattern("r_pentomino", [  # famous methuselah
                [0, 1, 1],
                [1, 1, 0],
                [0, 1, 0],
            ]),
        ]

        return {p.name: p for p in patterns}

    # ---------------------------
    # Transform utilities
    # ---------------------------

    @staticmethod
    def _copy_grid(g: Grid) -> Grid:
        return [row[:] for row in g]

    @staticmethod
    def _rotate90(g: Grid) -> Grid:
        # Rotate clockwise
        return [list(row) for row in zip(*g[::-1])]

    @staticmethod
    def _flip_h(g: Grid) -> Grid:
        # Horizontal flip (mirror left-right)
        return [row[::-1] for row in g]

    def _random_transform(self, g: Grid) -> Grid:
        # Randomly rotate 0/90/180/270 and optionally mirror.
        rotations = random.randint(0, 3)
        for _ in range(rotations):
            g = self._rotate90(g)

        if random.choice([True, False]):
            g = self._flip_h(g)

        return g


In [38]:
def main():
    game = GameOfLife(101, 82)
    shapes = Shapes(enable_transforms=True)

    # Option A: Randomly choose between blinker or glider
    #pat = shapes.random_blinker_or_glider()
    #game.set_pattern(pat.cells, 15, 15)
    #print("Placed:", pat.name)

    # Option B: Randomly choose from ALL patterns
    #pat = shapes.random()
    #game.set_pattern(pat.cells, 15, 15)

    # Option C: Pick a specific shape by name
    pat = shapes.get("pulsar")
    game.set_pattern(pat.cells, 30, 20)

    try:
        generation = 0
        while True:
            game.display()
            print(f"Generation: {generation} | Pattern: {pat.name}")
            generation += 1
            game.update()
            time.sleep(0.1)
    except KeyboardInterrupt:
        print("\nSimulation stopped by user")


In [39]:
if __name__ == "__main__":
    main()